In [9]:
# Import Stuff
import numpy as np
import os

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
# Reading Traning Data

dataImage = np.array([])
dataLabel = np.array([[]])
testImage = np.array([])
for i in range(1,6):
    rawData = unpickle('data/cifar-10-batches-py/data_batch_{}'.format(i))
    if( i==1 ): testImage = np.array( rawData[b'data'][0] )
    dataLabel = np.append(dataLabel, rawData[b'labels'])
    dataImage = np.append(dataImage, rawData[b'data'])
# dataImage
dataImage = dataImage.reshape(50000, -1)
dataLabel = dataLabel.astype('int32')
print(dataLabel.shape)
print(dataImage.shape)
print('Is data correct: ', 1 - np.sum(dataImage[0]-testImage))


(50000,)
(50000, 3072)
Is data correct:  1.0


In [4]:
# Reading Testing Data

rawData = unpickle('data/cifar-10-batches-py/test_batch')
dataTestImage = np.array(rawData[b'data'])
dataTestLabel = np.array(rawData[b'labels'])
dataTestLabel = dataTestLabel.astype('int32')
print(dataTestLabel.shape)
print(dataTestImage.shape)

(10000,)
(10000, 3072)


In [5]:
# Find K nearest Image

def kNearestImage(image):
    trainImageCount = dataImage.shape[0]
    result = []
    res = []
    for i in range(trainImageCount):
        trainImage = dataImage[i]
        diff  = np.sum( np.absolute( image - trainImage ) )
        res.append(diff)
    for j in range(K):
        currMinPos = 0
        for k in range(trainImageCount):
            if( (dataLabel[k], res[k]) in result ): continue
            if( res[k]<res[currMinPos] ): currMinPos = k
        result.append((dataLabel[currMinPos], res[currMinPos]))
    return result

def judge(resutlOfKNN):
    currMax = 0
    count = [0]*10
    for result in resutlOfKNN:
        count[ result[0] ] =  count[ result[0] ]+1
        currMax = max(currMax, count[ result[0] ])
    for i in range(10):
        if( count[i]==currMax ): return i
    return -1  
    
def predict(image):
    knn = kNearestImage(image)
    return judge(knn)

In [10]:
# RUN

K = 5 # Setting K: Find K closest image
testSize = 200 # MAX 10000

hits = 0
miss = 0
for i in range(testSize):
    predictLabel = predict( dataTestImage[i] )
    actualLabel = dataTestLabel[i]
    if( predictLabel == actualLabel ):
        hits = hits+1
    else:
        miss = miss+1
print(hits, miss, "{}%".format( 1.0*hits/testSize*100 ))



74 126 37.0%
